# Построение карты глубин с помощью opencv SGBM

Данный практикум демонстрирует работу алгоритма построения плотной карты глубин.
Исходными данными являются съемки со стереокамеры, установленной на трамвае (`data/images`).

Калибровка проведена и ее результаты лежат в `data/calib_matrices`

## Зачитывание входных изображений

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline


images_dir = '../data/depth_map/images'
os.listdir(images_dir)

In [ ]:
# Отделяет идентификатор камеры от имени файла
def parse_image_name(f):
    parts = f.split('.')
    image_id = '.'.join(parts[:3] + parts[4:-1])
    camera_id = parts[3]
    return (image_id, camera_id)

# Выберем только картинки
image_files = [f for f in os.listdir(images_dir) if os.path.splitext(f) == '.bmp']

# Получим уникальные имена пар картинок
image_ids = ['%02d' % i for i in range(5)]
left_name  = '_left'
right_name = '_righ'

# Зачитаем пары входных изображений
def read_image_pair(i, images_dir):  
    l_fname = os.path.join(images_dir, i + left_name + '.bmp')
    r_fname = os.path.join(images_dir, i + right_name + '.bmp')
    print(l_fname, r_fname)
    l = cv2.imread(l_fname)
    r = cv2.imread(r_fname)
    
    # OpenCV зачитывает изображения в формате BGR, переведем иъ в RGB чтобы они
    # нормально отображались pyplot
    l = cv2.cvtColor(l, cv2.COLOR_BGR2RGB)
    r = cv2.cvtColor(r, cv2.COLOR_BGR2RGB)
    return (l, r)

image_pairs = [read_image_pair(i, images_dir) for i in image_ids]

# Отобразим зачитанные пары изображений
def show_image_pairs(image_pairs):
    for l, r in image_pairs:
        plt.figure(figsize=(17,13))
        plt.subplot(121)
        plt.imshow(l)
        plt.subplot(122)
        plt.imshow(r)
        plt.show()
        
show_image_pairs(image_pairs)


## Зачитывание параметров ректификации

Нам потребуются: матрицы проекции обеих камер M1, M2, и их коэффициенты радиальной дисторсии D1, D2.

Так же понадобятся откалиброванные параметры взаимного размещения матриц: R, T

In [ ]:
calib_dir = '../data/depth_map/calib_matrices'

M1 = np.loadtxt(os.path.join(calib_dir, 'M1'))
M2 = np.loadtxt(os.path.join(calib_dir, 'M2'))
D1 = np.loadtxt(os.path.join(calib_dir, 'D1'))
D2 = np.loadtxt(os.path.join(calib_dir, 'D2'))
R =  np.loadtxt(os.path.join(calib_dir, 'R'))
T =  np.loadtxt(os.path.join(calib_dir, 'T'))

# Изображение хранится в np.array в порядке HWC.
# Получим размер изображения (W, H):
sz = image_pairs[0][0].shape[1::-1]

# Проверяем зачитанные параметры
print('Left camera intrinsics:', M1, D1)
print('Right camera intrinsics', M2, D2)
print('Stereo Extrinsics:', R, T)
print('baseline is %.2f meters' % np.linalg.norm(T))

Для ректификации необходимо рассчитать проективные ректифицирующие преобразования R1, R2 и новые матрицы камер P1, P2.

In [ ]:
# Теперь рассчитаем ректифицирующие преобразования
R1, R2, P1, P2, Q, roi1, roi2 = \
    cv2.stereoRectify(M1, D1, M2, D2, sz, R, T, alpha=0)
lmap1, lmap2 = cv2.initUndistortRectifyMap(M1, D1, R1, P1, sz, cv2.CV_32FC1)
rmap1, rmap2 = cv2.initUndistortRectifyMap(M2, D2, R2, P2, sz, cv2.CV_32FC1)

def get_rectified_pair(l, r):
    rect_l = cv2.remap(l, lmap1, lmap2, cv2.INTER_LINEAR)
    rect_r = cv2.remap(r, rmap1, rmap2, cv2.INTER_LINEAR)
    return rect_l, rect_r

rectified_images = [get_rectified_pair(l, r) for l,r in image_pairs]

show_image_pairs(rectified_images)


## Применение стерео восстановления

Описание параметров можно найти в [документации OpenCV](https://docs.opencv.org/3.2.0/d2/d85/classcv_1_1StereoSGBM.html#a58d7fac7a70b0a91ec15a46b5bb12e14)

Поиграйтесь с параметрами, чтобы найти наилучшее восстановление.

In [ ]:
# Параметры для карты глубин
min_d = 4
max_d = 16 * 9
block_size = 3
p1 = 100
p2 = 1000
disp12MaxDiff = 1
preFilterCap = 2
uniquenessRatio = 5 
speckleWindowSize = 400# 50  
speckleRange = 200# 2
mode = cv2.StereoSGBM_MODE_HH

# Объект-алгоритм SGBM

sbgm = cv2.StereoSGBM_create(
    min_d,
    max_d,
    block_size,
    p1,
    p2,
    disp12MaxDiff,
    preFilterCap,
    uniquenessRatio,
    speckleWindowSize,
    speckleRange,
    mode)

disparities = [sbgm.compute(l,r) for l,r in rectified_images]
disp_pairs = [(d, p[0]) for d, p in zip(disparities, rectified_images)]

show_image_pairs(disp_pairs)

## Контрольные вопросы

1. Почему картина карты глубин "обрезана" слева?
2. Чем объяснить темную область сзади человека на последнем изображении?
3. Чем объяснить артефакты карты глубин на outdoor изображениях?
4. Чем объяснить артефакты карты глубит на нижней части изображений?
5. Оцените теоретическую дальность восстановления стерео

In [ ]:
cv2.__version__